 Heroes of Pymoli



Congratulations! After a lot of hard work in the data munging mines, you've landed a job as Lead Analyst for an independent gaming company. You've been assigned the task of analyzing the data for their most recent fantasy game Heroes of Pymoli.

Like many others in its genre, the game is free-to-play, but players are encouraged to purchase optional items that enhance their playing experience. As a first task, the company would like you to generate a report that breaks down the game's purchasing data into meaningful insights.

Your final report should include each of the following:


In [1]:
#import various dependencies
import pandas as pd 
import csv
import numpy as np

In [2]:
#read csv data
path = "purchase_data.csv"
purchase_df=pd.read_csv(path)
purchase_df.head(4)

,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price
0,0,Lisim78,20,Male,108,"Extraction, Quickblade Of Trembling Hands",3.53
1,1,Lisovynya38,40,Male,143,Frenzied Scimitar,1.56
2,2,Ithergue48,24,Male,92,Final Critic,4.88
3,3,Chamassasya86,24,Male,100,Blindscythe,3.27


### Player Count

* Total Number of Players


In [3]:
#find the total number of players
player= len(purchase_df["SN"].value_counts())
player_count=pd.DataFrame([player], columns = [" Total  Players Count"])
player_count


,Total Players Count
0,576


### Purchasing Analysis (Total)

* Number of Unique Items
* Average Purchase Price
* Total Number of Purchases
* Total Revenue

In [4]:
#average price 
Avgprice = purchase_df["Price"].mean()
Number_of_purchases = len(purchase_df["Item Name"])

#unique items 
Unique_items = len(purchase_df["Item Name"].unique())

#total revenue
Sumrev =purchase_df["Price"].sum()

#create a df to hold items 
Purchasing_Analysis_Total = pd.DataFrame({"Number of Unique Items": [Unique_items],
                                           "Average Price": [Avgprice],
                                           "Number of Purchases": [Number_of_purchases],
                                           "Total Revenue": [Sumrev]})
#formatting data frame 
Purchasing_Analysis_Total["Average Price"] = Purchasing_Analysis_Total["Average Price"].map("${:.2f}".format)
Purchasing_Analysis_Total["Total Revenue"] = Purchasing_Analysis_Total["Total Revenue"].map("${:.2f}".format)
Purchasing_Analysis_Total = Purchasing_Analysis_Total[["Number of Unique Items", "Average Price", "Number of Purchases", "Total Revenue"]]

Purchasing_Analysis_Total

,Number of Unique Items,Average Price,Number of Purchases,Total Revenue
0,179,$3.05,780,$2379.77


### Gender Demographics

* Percentage and Count of Male Players
* Percentage and Count of Female Players
* Percentage and Count of Other / Non-Disclosed


In [5]:
#percentage + count of male players
sex = purchase_df[["SN","Gender"]]
sex = sex.drop_duplicates()
counts = sex["Gender"].value_counts()
#percentages
overall_counts = [counts[0],counts[1],counts[2]]
percentages = [round((counts[0]/player)*100,2),round((counts[1]/player)*100,2),round((counts[2]/player)*100,2)]

#making data frame + index
sex_example = pd.DataFrame({ "Percentage of Players": percentages,"Total Count": overall_counts
})
sex_example.index = (["Male", "Female", "Other / Non-Disclosed"])
sex_example


,Percentage of Players,Total Count
Male,84.03,484
Female,14.06,81
Other / Non-Disclosed,1.91,11


### Purchasing Analysis (Gender)

* The below each broken by gender
  * Purchase Count
  * Average Purchase Price
  * Total Purchase Value
  * Average Purchase Total per Person by Gender


In [6]:
# Group by Gender
sex = purchase_df.groupby(["Gender"])

# Data calculations
purch_Count = sex["SN"].count()

purch_Price = sex["Price"].mean()

purch_Value = sex["Price"].sum()

# sorting data by deleting all duplicates
duplicate = purchase_df.drop_duplicates(keep="first" ,subset="SN")
group_duplicate = duplicate.groupby(["Gender"])
#sorted data
purch_sorted= (sex["Price"].sum() / group_duplicate["SN"].count())

#dataframe
Purch_Anal_Gen = pd.DataFrame({"Purchase Count": purch_Count,
                              "Average Purchase Price": purch_Price,
                              "Total Purchase Value": purch_Value,
                              "Average Purchase Total per Person by Gender": purch_sorted})
#formatting
Purch_Anal_Gen["Average Purchase Price"] = Purch_Anal_Gen["Average Purchase Price"].map("${:.2f}".format)
Purch_Anal_Gen["Total Purchase Value"] = Purch_Anal_Gen["Total Purchase Value"].map("${:.2f}".format)
Purch_Anal_Gen["Average Purchase Total per Person by Gender"] = Purch_Anal_Gen["Average Purchase Total per Person by Gender"].map("${:.2f}".format)
Purch_Anal_Gen = Purch_Anal_Gen[["Purchase Count", "Average Purchase Price", "Total Purchase Value", "Average Purchase Total per Person by Gender"]]
Purch_Anal_Gen






,Purchase Count,Average Purchase Price,Total Purchase Value,Average Purchase Total per Person by Gender
Gender,,,,
Female,113,$3.20,$361.94,$4.47
Male,652,$3.02,$1967.64,$4.07
Other / Non-Disclosed,15,$3.35,$50.19,$4.56


### Age Demographics

* The below each broken into bins of 4 years (i.e. &lt;10, 10-14, 15-19, etc.)
  * Purchase Count
  * Average Purchase Price
  * Total Purchase Value
  * Average Purchase Total per Person by Age Group

In [21]:
#Bins for Age Demographics
bins = [0]
bin_value = ["<10"]
max_age = purchase_df["Age"].max()

for x in range(2,int(max_age/5)+2):
    bins = bins + [(x*5)-1]

for x in range(1,len(bins)-2):
    bin_value = bin_value + [str(bins[x]+1)+"-"+str(bins[x+1])]

bin_value = bin_value + [str(max_age) + "+"]
#print(bins)
#print(bin_value)

#Age Demographics - need to group by SN.. add back in age, gender.. groupby age group (count and percentage)
purchase_df["Age Group"] = pd.cut(purchase_df["Age"],bins,labels = bin_value)

#Purchasing Analysis (Age)
age_group = purchase_df.groupby(["Age Group"])
age_count = age_group.size()
age_avg_price = round(age_group["Price"].mean(),2)
age_tot_value = age_group["Price"].sum()
age_norm_tot = round(age_tot_value/age_count,2)
age_dataframe = pd.DataFrame({"Purchase Count":age_count,
                             "Average Purchase Price": age_avg_price,
                             "Total Purchase Value": age_tot_value,
                             "Average Purchase Total per Person by Age Group": age_norm_tot})

agedemographics= age_dataframe[["Purchase Count","Average Purchase Price", "Total Purchase Value", "Average Purchase Total per Person by Age Group"]]
agedemographics

,Purchase Count,Average Purchase Price,Total Purchase Value,Average Purchase Total per Person by Age Group
Age Group,,,,
<10,23,3.35,77.13,3.35
10-14,28,2.96,82.78,2.96
15-19,136,3.04,412.89,3.04
20-24,365,3.05,1114.06,3.05
25-29,101,2.90,293.00,2.90
30-34,73,2.93,214.00,2.93
35-39,41,3.60,147.67,3.60
40-44,12,3.04,36.54,3.04
45+,1,1.70,1.70,1.70


In [20]:
max_age

45

### Top Spenders

* Identify the the top 5 spenders in the game by total purchase value, then list (in a table):
  * SN
  * Purchase Count
  * Average Purchase Price
  * Total Purchase Value


In [ ]:
#define top spender 
top_spenders = purchase_df['Item ID'].groupby(purchase_df['SN']).count()

# set the dataframe for top spender 
top_spenders= pd.DataFrame(data=top_spenders)
top_spenders.columns = ['Purchase Count']

top_spenders['Average Purchase Price'] = round(purchase_df['Price'].groupby(purchase_df['SN']).mean(),2)
top_spenders['Total Purchase Value'] = purchase_df['Price'].groupby(purchase_df['SN']).sum()


top_spenders.sort_values(by=['Total Purchase Value'], ascending=False, inplace=True)

top_spenders['Average Purchase Price'] = top_spenders['Average Purchase Price'].map('${:,.2f}'.format)
top_spenders['Total Purchase Value'] = top_spenders['Total Purchase Value'].map('${:,.2f}'.format)

top_spenders.head()

### Most Popular Items

* Identify the 5 most popular items by purchase count, then list (in a table):
  * Item ID
  * Item Name
  * Purchase Count
  * Item Price
  * Total Purchase Value


In [ ]:
#define most items 
most_items = purchase_df.groupby(['Item ID', 'Item Name','Price'])['Price'].agg(['count','sum'])
most_items.columns = ['Purchase Count', 'Total Purchase Value']

# set the index back at zero 
most_items.reset_index(inplace=True)
most_items.set_index(['Item ID','Item Name'] ,inplace=True)

most_items = most_items[['Purchase Count', 'Price', 'Total Purchase Value']]

#Arrange the purchase count column in descending order
most_popular_items = most_items.sort_values(by='Purchase Count', ascending=False)

#formatting
most_popular_items['Price'] = most_popular_items['Price'].map('${:,.2f}'.format)
most_popular_items['Total Purchase Value'] = most_popular_items['Total Purchase Value'].map('${:,.2f}'.format)

most_popular_items.head()

### Most Profitable Items

* Identify the 5 most profitable items by total purchase value, then list (in a table):
  * Item ID
  * Item Name
  * Purchase Count
  * Item Price
  * Total Purchase Value

In [ ]:
#in descending order 
most_profitable_items = most_items.sort_values(by='Total Purchase Value', ascending=False)

#the data frame 
most_profitable_items['Price'] = most_profitable_items['Price'].map('${:,.2f}'.format)
most_profitable_items['Total Purchase Value'] = most_profitable_items['Total Purchase Value'].map('${:,.2f}'.format)
most_profitable_items.head()

#END